In [ ]:
import s3fs
fs = s3fs.S3FileSystem()

files=fs.ls('s3://cleanmsds18015/clean/')

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import boto3
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

In [ ]:
df_list=[]
for file in files:
    #print(file)
    df_list.append(pd.read_csv('s3://'+file))

In [ ]:
df = pd.concat(df_list)

In [ ]:
df.shape

(704867, 8)

In [ ]:
Xtrain_df,Xtest_df,ytrain_df,ytest_df=train_test_split(df['clean_text'],df['polarity'],
                                                 test_size=0.3)
Xtrain=Xtrain_df.values
Xtest=Xtest_df.values
ytrain=ytrain_df.values
ytest=ytest_df.values

In [ ]:
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
unigram_vectorizer.fit(Xtrain)
X_train_unigram = unigram_vectorizer.transform(Xtrain)
unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)
X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)


X_test_unigram = unigram_vectorizer.transform(Xtest)
unigram_tf_idf_transformer.fit(X_test_unigram)
X_test_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_test_unigram)

In [ ]:
ytrain=0.5+ytrain/2
ytrain=np.round(ytrain)
ytrain=ytrain.astype('int')

ytest=0.5+ytest/2
ytest=np.round(ytest)
ytest=ytest.astype('int')

In [ ]:
clf = SGDClassifier()
clf.fit(X_train_unigram_tf_idf, ytrain)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
train_pred=clf.score(X_train_unigram_tf_idf, ytrain)
test_pred=clf.score(X_test_unigram_tf_idf, ytest)
print('Training accuracy:',100*train_pred)
print('Test accuracy:',100*test_pred)

Training accuracy: 86.38545133216864
Test accuracy: 86.55733208487617


In [ ]:
test_pred=clf.predict(X_test_unigram_tf_idf)

In [ ]:
neg=Xtest_df[test_pred==0]
pos=Xtest_df[test_pred==1]
neg = neg.str.lower()
pos = pos.str.lower()
neg= neg.str.replace('.','')
neg= neg.str.replace('#','')
pos= pos.str.replace('.','')
pos= pos.str.replace('#','')
neg_count=neg.str.split(expand=True).stack().value_counts()
pos_count=pos.str.split(expand=True).stack().value_counts()
tempkeys=[]
for key in neg_count.keys():
    if key in stopwords.words('english'):
        tempkeys.append(key)
    if key in pos_count.keys():
        if pos_count[key]>1000:
            tempkeys.append(key)
neg_count_new=neg_count.drop(labels=tempkeys)
tempkeys=[]
for key in pos_count.keys():
    if key in stopwords.words('english'):
        tempkeys.append(key)
    if key in neg_count.keys():
        if neg_count[key]>1000:
            tempkeys.append(key)
pos_count_new=pos_count.drop(labels=tempkeys)
pos_count_new=pd.DataFrame({'word':pos_count_new.index, 'frequency':pos_count_new.values})
neg_count_new=pd.DataFrame({'word':neg_count_new.index, 'frequency':neg_count_new.values})

file_name ="positive_words_pred.csv" 
pos_count_new.to_csv(file_name)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'cleanmsds18015', file_name)

file_name ="negative_words_pred.csv" 
neg_count_new.to_csv(file_name)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'cleanmsds18015', file_name)

In [ ]:
'''
file_name ="cleancombined.csv" 
df.to_csv(file_name)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'cleanmsds18015', file_name)
'''

'\nfile_name ="cleancombined.csv" \ndf.to_csv(file_name)\ns3 = boto3.resource(\'s3\')\ns3.meta.client.upload_file(file_name, \'cleanmsds18015\', file_name)\n'

In [ ]:
neg=df[np.round(df['polarity'])==-1.0]
pos=df[np.round(df['polarity'])==1.0]
neu=df[np.round(df['polarity']).astype('int')==0]

In [ ]:
neg['clean_text'] = neg['clean_text'] .str.lower()
pos['clean_text'] = pos['clean_text'] .str.lower()
neu['clean_text'] = neu['clean_text'] .str.lower()
neg['clean_text'] = neg['clean_text'] .str.replace('.','')
neg['clean_text'] = neg['clean_text'] .str.replace('#','')
pos['clean_text'] = pos['clean_text'] .str.replace('.','')
pos['clean_text'] = pos['clean_text'] .str.replace('#','')

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [ ]:
neg_count=neg.clean_text.str.split(expand=True).stack().value_counts()
pos_count=pos.clean_text.str.split(expand=True).stack().value_counts()
neu_count=neu.clean_text.str.split(expand=True).stack().value_counts()

In [ ]:
tempkeys=[]
for key in neg_count.keys():
    if key in stopwords.words('english'):
        tempkeys.append(key)
    if key in pos_count.keys():
        if pos_count[key]>1000:
            tempkeys.append(key)
neg_count_new=neg_count.drop(labels=tempkeys)

In [ ]:
tempkeys=[]
for key in pos_count.keys():
    if key in stopwords.words('english'):
        tempkeys.append(key)
    if key in neg_count.keys():
        if neg_count[key]>1000:
            tempkeys.append(key)
pos_count_new=pos_count.drop(labels=tempkeys)

In [ ]:
tempkeys=[]
for key in neu_count.keys():
    if key in stopwords.words('english'):
        tempkeys.append(key)
neu_count_new=neu_count.drop(labels=tempkeys)

In [ ]:
neu_count_new=pd.DataFrame({'word':neu_count_new.index, 'frequency':neu_count_new.values})
pos_count_new=pd.DataFrame({'word':pos_count_new.index, 'frequency':pos_count_new.values})
neg_count_new=pd.DataFrame({'word':neg_count_new.index, 'frequency':neg_count_new.values})

In [ ]:
neu_count_new.head(2)

,word,frequency
0,coronavirus,343586
1,#coronavirus,112467


In [ ]:
file_name ="neutral_words.csv" 
neu_count_new.to_csv(file_name)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'cleanmsds18015', file_name)

file_name ="positive_words.csv" 
pos_count_new.to_csv(file_name)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'cleanmsds18015', file_name)

file_name ="negative_words.csv" 
neg_count_new.to_csv(file_name)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'cleanmsds18015', file_name)

In [ ]:
'''
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
lmtzr.lemmatize('I.')
'''

"\nfrom nltk.stem.wordnet import WordNetLemmatizer\nlmtzr = WordNetLemmatizer()\nlmtzr.lemmatize('I.')\n"

In [ ]:
#import nltk
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True